In [1]:
import os
import json

In [3]:
class FileStorage():

    def __init__(self,filename):
        self.filename = filename

    def read(self):
        if os.path.exists(self.filename):
            with open(self.filename) as file:
                data = json.load(file)
                return data
        else:
            return {}
        
    def save(self,data):#write data like json file
        try:
            old_data = self.read()
            if len(old_data.keys()) == 0:
                old_data["tweets"] = []
            old_data["tweets"].append(data)
            jsondata = json.dumps(old_data, indent=4, skipkeys=True, sort_keys=True)
            fd = open(self.filename, 'w')
            fd.write(jsondata)
            fd.close()
            print (self.filename + " ha sido escrito exitosamente")
        except Exception as e:
            print (e)
            print ('ERROR writing', self.filename)



In [6]:
#remove stop words
import nltk
from nltk.corpus import stopwords
import re
import sys
import unicodedata


def filter_stop_words(text):
    stop_words_list = stopwords.words('spanish')
    stop_words_list += stopwords.words('english')
    text_filtered = [word for word in text.split() if word.lower() not in stop_words_list]
    return text_filtered

def process(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"(https?)\S+","",tweet)#remove urls
    #tweet = re.sub(r"(\B#)\w*","",tweet)#remove hashtags
    tweet = re.sub(r"(\B@)\w*","",tweet)#remove mentions
    tweet = re.sub("\n","",tweet)#remove lines separate
    tweet = unicodedata.normalize('NFKD', tweet).encode('ASCII', 'ignore')#remove accents
    tweet = re.sub('[^a-zA-Z ]+', ' ', tweet.decode('ASCII'))#remove punctuactions
    text_no_stop_words = filter_stop_words(tweet)
    tokens = [token for token in text_no_stop_words if len(token) > 2 ]
    tweet = " ".join(tokens)
    return tweet



In [4]:
file_worker = FileStorage("buenos_aires.json")
data = file_worker.read()
print(data['tweets'][0])

{'contributors': None, 'coordinates': None, 'created_at': 'Sat Nov 28 14:51:38 +0000 2020', 'display_text_range': [9, 42], 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': [{'id': 124599878, 'id_str': '124599878', 'indices': [0, 8], 'name': 'Tartu', 'screen_name': 'TartuTV'}]}, 'favorite_count': 0, 'favorited': False, 'filter_level': 'low', 'geo': None, 'id': 1332698671285198855, 'id_str': '1332698671285198855', 'in_reply_to_screen_name': 'TartuTV', 'in_reply_to_status_id': 1332313427348434944, 'in_reply_to_status_id_str': '1332313427348434944', 'in_reply_to_user_id': 124599878, 'in_reply_to_user_id_str': '124599878', 'is_quote_status': False, 'lang': 'es', 'place': {'attributes': {}, 'bounding_box': {'coordinates': [[[-58.531792, -34.674453], [-58.531792, -34.534177], [-58.353494, -34.534177], [-58.353494, -34.674453]]], 'type': 'Polygon'}, 'country': 'Argentina', 'country_code': 'AR', 'full_name': 'Ciudad Autónoma de Buenos Aires, Argentina', 'id': '018f1cde6b

In [33]:
hashtags_count = {}

In [34]:
#for tweet in data['tweets'][8:]:
for tweet in data['tweets']:
#    print(tweet['entities']['hashtags'])
    tweet_hashtags = tweet['entities']['hashtags']
    hashtags_list = [hashtag['text'].lower() for hashtag in tweet_hashtags]
    hashtags_list.sort()
#    print(hashtags_list)
    hashtags_comb = list(itertools.combinations(hashtags_list, 2))
#    print(hashtags_comb)
    for comb in hashtags_comb:
        if comb not in hashtags_count:
            hashtags_count[comb] = 1
        else:
            hashtags_count[comb] += 1
#    print(hashtags_count)
#    break

In [35]:
hashtags_count

{('diasdelluvia', 'efectos'): 1,
 ('diasdelluvia', 'efectoscluster'): 1,
 ('diasdelluvia', 'pedaloftheday'): 1,
 ('efectos', 'efectoscluster'): 1,
 ('efectos', 'pedaloftheday'): 1,
 ('efectoscluster', 'pedaloftheday'): 1,
 ('bahraingp', 'f1'): 2,
 ('bahraingp', 'leclerc'): 1,
 ('bahraingp', 'russell'): 1,
 ('bahraingp', 'sainz'): 1,
 ('bahraingp', 'stroll'): 1,
 ('bahraingp', 'vettel'): 1,
 ('f1', 'leclerc'): 1,
 ('f1', 'russell'): 1,
 ('f1', 'sainz'): 1,
 ('f1', 'stroll'): 1,
 ('f1', 'vettel'): 1,
 ('leclerc', 'russell'): 1,
 ('leclerc', 'sainz'): 1,
 ('leclerc', 'stroll'): 1,
 ('leclerc', 'vettel'): 1,
 ('russell', 'sainz'): 1,
 ('russell', 'stroll'): 1,
 ('russell', 'vettel'): 1,
 ('sainz', 'stroll'): 1,
 ('sainz', 'vettel'): 1,
 ('stroll', 'vettel'): 1,
 ('bigass', 'onlyfans'): 1,
 ('bigass', 'onlyfansgirl'): 1,
 ('bigass', 'rt'): 1,
 ('bigass', 'suscribete'): 1,
 ('onlyfans', 'onlyfansgirl'): 1,
 ('onlyfans', 'rt'): 2,
 ('onlyfans', 'suscribete'): 1,
 ('onlyfansgirl', 'rt'): 1,
 (

In [37]:
f = open("hashtags_network.csv", "w")
f.write('source,target,weight\n')
for comb in hashtags_count:
    count = str(hashtags_count[comb])
    hashtag1 = comb[0]
    hashtag2 = comb[1]
    print(hashtag1, hashtag2, count)
    f.write(hashtag1 + ',' + hashtag2 + ',' + count + '\n')
f.close()

diasdelluvia efectos 1
diasdelluvia efectoscluster 1
diasdelluvia pedaloftheday 1
efectos efectoscluster 1
efectos pedaloftheday 1
efectoscluster pedaloftheday 1
bahraingp f1 2
bahraingp leclerc 1
bahraingp russell 1
bahraingp sainz 1
bahraingp stroll 1
bahraingp vettel 1
f1 leclerc 1
f1 russell 1
f1 sainz 1
f1 stroll 1
f1 vettel 1
leclerc russell 1
leclerc sainz 1
leclerc stroll 1
leclerc vettel 1
russell sainz 1
russell stroll 1
russell vettel 1
sainz stroll 1
sainz vettel 1
stroll vettel 1
bigass onlyfans 1
bigass onlyfansgirl 1
bigass rt 1
bigass suscribete 1
onlyfans onlyfansgirl 1
onlyfans rt 2
onlyfans suscribete 1
onlyfansgirl rt 1
onlyfansgirl suscribete 1
rt suscribete 1
avz buensábado 1
noticieros qepd 1
ikiaşıkkalp sençalkapımı 8
fedua fisu 1
buensabado lluviadebendiciones 1
lg samsung 1
lg springer 1
samsung springer 1
retweet sabadodeganarseguidores 1
retweet siguemeytesigo 1
sabadodeganarseguidores siguemeytesigo 1
ad10s diegoeterno 4
caretas maradona 1
maradona modocarr